In [1]:
!pip install evaluate torchmetrics -q

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset
from sklearn.metrics import mean_squared_error
import evaluate
from torchmetrics.functional.regression import cosine_similarity
from torchmetrics.functional.regression import mean_squared_error , pearson_corrcoef

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

2024-06-11 08:02:28.316433: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-11 08:02:28.316599: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-11 08:02:28.451293: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/all-nli-dataset/pair-class_dev.tsv
/kaggle/input/all-nli-dataset/pair-class_train.tsv
/kaggle/input/all-nli-dataset/triplet_dev.tsv
/kaggle/input/all-nli-dataset/stsb_train.tsv
/kaggle/input/all-nli-dataset/pair_dev.tsv
/kaggle/input/all-nli-dataset/pair_test.tsv
/kaggle/input/all-nli-dataset/triplet_test.tsv
/kaggle/input/all-nli-dataset/pair-class_test.tsv
/kaggle/input/all-nli-dataset/pair-score_train.tsv
/kaggle/input/all-nli-dataset/pair-score_dev.tsv
/kaggle/input/all-nli-dataset/pair_train.tsv
/kaggle/input/all-nli-dataset/pair-score_test.tsv
/kaggle/input/all-nli-dataset/stsb_test.tsv
/kaggle/input/all-nli-dataset/stsb_dev.tsv
/kaggle/input/all-nli-dataset/triplet_train.tsv


In [3]:
dataset = load_dataset("csv", data_files={
    "train": "/kaggle/input/all-nli-dataset/stsb_train.tsv",
    "dev": "/kaggle/input/all-nli-dataset/stsb_dev.tsv",
    "test": "/kaggle/input/all-nli-dataset/stsb_test.tsv"
}, delimiter="\t")


Generating train split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
device

device(type='cuda')

In [6]:
class STSBDataset(Dataset):
    def __init__(self, sentences1, sentences2, scores, tokenizer, max_length):
        self.sentences1 = sentences1
        self.sentences2 = sentences2
        self.scores = scores
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.sentences1)

    def __getitem__(self, idx):
        sent1 = self.sentences1[idx]
        sent2 = self.sentences2[idx]
        score = self.scores[idx]

        encoded_sent1 = self.tokenizer(sent1, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        encoded_sent2 = self.tokenizer(sent2, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        
        input1 = {key: val.squeeze(0).to(device) for key, val in encoded_sent1.items()}
        input2 = {key: val.squeeze(0).to(device) for key, val in encoded_sent2.items()}

        return (input1, input2), torch.tensor(score, dtype=torch.float, device=device)

In [7]:
train_sentences1 = dataset['train']['sentence1']
train_sentences2 = dataset['train']['sentence2']
train_scores = dataset['train']['score']

val_sentences1 = dataset['dev']['sentence1']
val_sentences2 = dataset['dev']['sentence2']
val_scores = dataset['dev']['score']


In [8]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
max_length = 128

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
train_dataset = STSBDataset(train_sentences1, train_sentences2, train_scores, tokenizer, max_length)
val_dataset = STSBDataset(val_sentences1, val_sentences2, val_scores, tokenizer, max_length)

In [10]:
train_loader = DataLoader(train_dataset, batch_size=512 * 4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=512 * 4, shuffle=False)

In [11]:
# class SentenceEmbeddingModel(nn.Module):
#     def __init__(self, hidden_dim, output_dim):
#         super(SentenceEmbeddingModel, self).__init__()
#         self.distilbert = AutoModel.from_pretrained('distilbert-base-uncased')
#         self.distilbert.to(device)
#         self.lstm = nn.LSTM(768, hidden_dim, batch_first=True)
#         self.lstm.flatten_parameters()
#         self.fc = nn.Linear(hidden_dim, output_dim)

#     def forward(self, input1, input2):
#         with torch.no_grad():
#             emb1 = self.distilbert(**input1).last_hidden_state[:, 0, :]
#             emb2 = self.distilbert(**input2).last_hidden_state[:, 0, :]
        
#         print(emb1.shape)
#         combined = torch.cat((emb1, emb2), dim=1)
#         print(combined.unsqueeze(1).shape)
#         lstm_out, _ = self.lstm(combined.unsqueeze(1))
#         lstm_out = lstm_out[:, -1, :]
#         output = self.fc(lstm_out)
#         return output.squeeze()
    
#     def flatten_lstm_parameters(self):
#         self.lstm.flatten_parameters()

class SentenceEmbeddingModel(nn.Module):
    def __init__(self):
        super(SentenceEmbeddingModel, self).__init__()
        self.distilbert = AutoModel.from_pretrained('distilbert-base-uncased')
        self.lstm = nn.LSTM(input_size=1536, hidden_size=768, num_layers=1, batch_first=True)
        self.fc = nn.Linear(768, 1)

    def forward(self, input1, input2):
        with torch.no_grad():
            emb1 = self.distilbert(**input1).last_hidden_state[:, 0, :]  # [batch_size, hidden_size]
            emb2 = self.distilbert(**input2).last_hidden_state[:, 0, :]  # [batch_size, hidden_size]

        # print(f"emb1 shape: {emb1.shape}")  # Should be [batch_size, hidden_size]
        # print(f"emb2 shape: {emb2.shape}")  # Should be [batch_size, hidden_size]

        combined = torch.cat((emb1, emb2), dim=1)  # [batch_size, hidden_size * 2]
        # print(f"combined shape: {combined.shape}")  # Should be [batch_size, hidden_size * 2]
        
        combined = combined.unsqueeze(1)  # [batch_size, 1, hidden_size * 2]
        # print(f"combined unsqueeze(1) shape: {combined.shape}")  # Should be [batch_size, 1, hidden_size * 2]

        lstm_out, _ = self.lstm(combined)  # [batch_size, 1, hidden_size]
        # print(f"lstm_out shape: {lstm_out.shape}")  # Should be [batch_size, 1, hidden_size]
        
        lstm_out = lstm_out[:, -1, :]  # [batch_size, hidden_size]
        # print(f"lstm_out[:, -1, :] shape: {lstm_out.shape}")  # Should be [batch_size, hidden_size]
        
        output = self.fc(lstm_out)  # [batch_size, 1]
        # print(f"output shape: {output.shape}")  # Should be [batch_size, 1]

        return output

    def flatten_lstm_parameters(self):
        self.lstm.flatten_parameters()

In [12]:
hidden_dim = 128
output_dim = 1
model = SentenceEmbeddingModel()

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [13]:
model.to(device)

SentenceEmbeddingModel(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (l

In [14]:
# Training loop
def train_model(model, train_loader, criterion, optimizer, num_epochs=1):
    # Set the model to training mode
    model.train()
#     model.to(device)
    model.flatten_lstm_parameters()

    for epoch in range(num_epochs):
        for (encoded_sent1, encoded_sent2), scores in train_loader:
            # Move inputs and targets to the same device as the model


            optimizer.zero_grad()
            outputs = model(encoded_sent1, encoded_sent2)
            loss = criterion(outputs, scores)
            loss.backward()
            optimizer.step()

            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


In [15]:
# Assuming model, train_loader, criterion, and optimizer are already defined
train_model(model, train_loader, criterion, optimizer, num_epochs=3)

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([2048])) that is different to the input size (torch.Size([2048, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/3], Loss: 0.4505
Epoch [1/3], Loss: 0.3063


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1653])) that is different to the input size (torch.Size([1653, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/3], Loss: 0.2261
Epoch [2/3], Loss: 0.1488
Epoch [2/3], Loss: 0.1045
Epoch [2/3], Loss: 0.0945
Epoch [3/3], Loss: 0.0973
Epoch [3/3], Loss: 0.1105
Epoch [3/3], Loss: 0.1335


In [16]:
predictions, true_scores = [], []

In [17]:
def evaluate_model(model, val_loader):
    model.eval()
    with torch.no_grad():
        for (input1, input2), scores in val_loader:
            outputs = model(input1, input2)
            predictions.extend(outputs.cpu().numpy())
            true_scores.extend(scores.cpu().numpy())

In [18]:
evaluate_model(model,val_loader)

In [19]:
pearson_corr_eval = evaluate.load('pearsonr')
mse_eval = evaluate.load('mse')

In [20]:
print("Test Data: ")
print(f"Pearson Correlation: {pearson_corr_eval.compute(predictions=predictions, references=true_scores)['pearsonr']}")
print(f"MSE: {mse_eval.compute(predictions=predictions, references=true_scores)['mse']}")
print(f"cosine_similarity: {cosine_similarity(torch.tensor(predictions).view(1, 1500), torch.tensor([true_scores]))}")

Test Data: 
Pearson Correlation: 0.07391903432552323
MSE: 0.1901072448440299
cosine_similarity: 0.8434118628501892


/tmp/ipykernel_24/2764345837.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:261.)
  print(f"cosine_similarity: {cosine_similarity(torch.tensor(predictions).view(1, 1500), torch.tensor([true_scores]))}")
